In [1]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import xarray as xr
%pylab inline
rcParams['savefig.dpi'] = 150
plt.close('all')
%pwd
plotme= True
figuredir = './figures'
%mkdir figures
from tabulate import tabulate
def save_fig(name,dpi=300, *args, **kwargs):
        plt.savefig(figuredir+name,dpi=dpi, bbox_inches='tight')

Populating the interactive namespace from numpy and matplotlib
mkdir: figures: File exists


/Users/pwolfram/miniconda2/lib/python2.7/site-packages/xarray/core/formatting.py:16: FutureWarning: The pandas.tslib module is deprecated and will be removed in a future version.
  from pandas.tslib import OutOfBoundsDatetime


In [2]:
ds = xr.open_dataset('transport.nc')

In [3]:
ds

<xarray.Dataset>
Dimensions:        (Time: 3720, nTransects: 15)
Coordinates:
  * Time           (Time) float64 0.04252 0.1233 0.2042 0.2877 0.3713 0.4549 ...
Dimensions without coordinates: nTransects
Data variables:
    TransectNames  (nTransects) |S64 'Drake Passage' 'Tasmania-Ant' ...
    Transport      (Time, nTransects) float64 146.8 162.7 152.9 -12.61 ...

In [4]:
ds.TransectNames

<xarray.DataArray 'TransectNames' (nTransects: 15)>
array(['Drake Passage', 'Tasmania-Ant', 'Africa-Ant', 'Antilles Inflow',
       'Mona Passage', 'Windward Passage', 'Florida-Cuba', 'Florida-Bahamas',
       'Indonesian Throughflow', 'Agulhas', 'Mozambique Channel',
       'Bering Strait', 'Lancaster Sound', 'Fram Strait', 'Robeson Channel'],
      dtype='|S64')
Dimensions without coordinates: nTransects

In [5]:
# filter out unneeded data

# observations (Positive values are north and eastward, see Ringler et al 2013).  References from zotero shared "observations" folder in ACME Land Ice-Ocean Coupling papers
obs = {'Drake Passage': {'mean': 147.5 , 'std dev': 27.5, 'ref': '\citep{donohue_mean_2016,nowlin_physics_1986}'}, 
       'Tasmania-Ant': {'mean': 157, 'std dev': 10, 'ref': '\citep{ganachaud_improved_2000,ganachaud_large-scale_2003}'},
       'Africa-Ant': {'mean': 150, 'std dev': 30, 'ref': 'Estimated from Drake and Tasmania-Ant passages from \citep{ganachaud_improved_2000,ganachaud_large-scale_2003}'},
       'Indonesian Throughflow': {'mean': -15, 'std dev': 4, 'ref': '\citep{sprintall_direct_2009}'},
       'Agulhas': {'mean': -70, 'std dev': 20, 'ref': '\citep{bryden_role_2001}' },
       'Mozambique Channel': {'mean': -16, 'std dev': 13, 'ref': '\citep{van_der_werf_comparison_2010}'},
       'Antilles Inflow': {'mean': -18.4, 'std dev': 4.7, 'ref': '\citep{johns_atlantic_2002, roemmich_circulation_1981}'},
       'Mona Passage': {'mean': -2.6, 'std dev': 1.2, 'ref': '\citep{johns_atlantic_2002, roemmich_circulation_1981}'},
       'Windward Passage': {'mean': 6.0, 'std dev': 3, 'ref': '\citep{johns_atlantic_2002, roemmich_circulation_1981}'},
       'Florida-Cuba': {'mean': 31, 'std dev': 1.5, 'ref': '\citep{johns_atlantic_2002, roemmich_circulation_1981}'},
       'Florida-Bahamas': {'mean': 31.5, 'std dev': 1.5, 'ref': '\citep{johns_atlantic_2002, roemmich_circulation_1981}'},
       'Bering Strait': {'mean': 0.8, 'std dev': 0.3, 'ref': '\citep{roach_direct_1995}'},
       'Lancaster Sound': {'mean': 0.8 , 'std dev': 0.3, 'ref': '\citep{prinsenberg_monitoring_2005}'},
       'Fram Strait': {'mean': -3, 'std dev': 3, 'ref': '\citep{schauer_arctic_2004}'},       
       'Robeson Channel': {'mean': -0.7 , 'std dev': .2 , 'ref': '\citep{maltrud_eddy_2005}'}
       }

In [6]:
def check_in_range(amean, astddev, bmean, bstddev):
    amax = amean + astddev
    amin = amean - astddev
    bmax = bmean + bstddev
    bmin = bmean - bstddev
    
    inrange = ((amin < bmax) and (amin > bmin)) or ((amax < bmax) and (amax > bmin))
    return inrange

# build the headers    
headers = []
headers.append('Transect location')
headers.append('EC60to30 (Sv)')
#headers.append('Result in range?')
headers.append('RRS18to6 (Sv)')
headers.append('Observation (Sv)')
headers.append('Observation reference')
#for atrans in ds.TransectNames:
#    headers.append(atrans.values)
    
table = []
# structure data into table format
for name, mean, stddev in zip(ds.TransectNames.values, ds.Transport.mean('Time').values, ds.Transport.std('Time').values):
    newrow = []
    newrow.append(str(name))
    newrow.append(u'%0.1f $\pm$ %0.1f'%(mean, stddev))
    newrow.append(u'X $\pm$ X')
    # observations
    obsmean = obs[str(name)]['mean']
    obsstd = obs[str(name)]['std dev']
    newrow.append(u'%0.1f $\pm$ %0.1f'%(obsmean,obsstd))
    #newrow.append(str(check_in_range(mean, stddev, obsmean, obsstd)))
    newrow.append(str(obs[str(name)]['ref']))
    table.append(newrow)

print 'Note: Africa-Ant completely estimated from a heuristic control volume analysis and could be removed from the table!'
print tabulate(table, headers=headers, floatfmt="0.1f", numalign="center") 

Note: Africa-Ant completely estimated from a heuristic control volume analysis and could be removed from the table!
Transect location       EC60to30 (Sv)    RRS18to6 (Sv)    Observation (Sv)    Observation reference
----------------------  ---------------  ---------------  ------------------  --------------------------------------------------------------------------------------------------------------
Drake Passage           83.8 $\pm$ 17.7  X $\pm$ X        147.5 $\pm$ 27.5    \citep{donohue_mean_2016,nowlin_physics_1986}
Tasmania-Ant            98.0 $\pm$ 20.0  X $\pm$ X        157.0 $\pm$ 10.0    \citep{ganachaud_improved_2000,ganachaud_large-scale_2003}
Africa-Ant              83.4 $\pm$ 17.9  X $\pm$ X        150.0 $\pm$ 30.0    Estimated from Drake and Tasmania-Ant passages from \citep{ganachaud_improved_2000,ganachaud_large-scale_2003}
Antilles Inflow         -15.9 $\pm$ 3.2  X $\pm$ X        -18.4 $\pm$ 4.7     \citep{johns_atlantic_2002, roemmich_circulation_1981}
Mona Passage

In [7]:
latextable = tabulate(table, headers=headers, floatfmt=".2f", numalign="center", tablefmt="latex_booktabs").replace('\\{','{').replace('\\}','}').replace('\\_','_').replace(u"\$",u"$").replace(u"\\textbackslash{}",u"\\")
print latextable
with open('transportstable.tex','w') as f:
    f.write(latextable)

\begin{tabular}{lllll}
\toprule
 Transect location      & EC60to30 (Sv)   & RRS18to6 (Sv)   & Observation (Sv)   & Observation reference                                                                                          \\
\midrule
 Drake Passage          & 83.8 $\pm$ 17.7 & X $\pm$ X       & 147.5 $\pm$ 27.5   & \citep{donohue_mean_2016,nowlin_physics_1986}                                                                  \\
 Tasmania-Ant           & 98.0 $\pm$ 20.0 & X $\pm$ X       & 157.0 $\pm$ 10.0   & \citep{ganachaud_improved_2000,ganachaud_large-scale_2003}                                                     \\
 Africa-Ant             & 83.4 $\pm$ 17.9 & X $\pm$ X       & 150.0 $\pm$ 30.0   & Estimated from Drake and Tasmania-Ant passages from \citep{ganachaud_improved_2000,ganachaud_large-scale_2003} \\
 Antilles Inflow        & -15.9 $\pm$ 3.2 & X $\pm$ X       & -18.4 $\pm$ 4.7    & \citep{johns_atlantic_2002, roemmich_circulation_1981}                                  